In [ ]:
# default_exp tef_core
from nbdev import *

# TEF core framework

> contains TEF framework

In [ ]:
#export
import math
import numpy as np
import xarray as xr
import logging

In [ ]:
#export
logging.basicConfig(level=logging.INFO,
                    filename="logger.TEF.GETM",
                    filemode="w",
                    format="%(asctime)-15s %(levelname)-8s %(message)s")
logger = logging.getLogger(__name__)

In [ ]:
#export
class TEF_object():
    """Total Exchange Flow (TEF) Object used for calculating TEF properties"""
    def __init__(self, filename = None, ds = None, **kwargs):

        # What happens if no filename is provided
        if not filename:
            if ds is None:
                self.ds = None
            else:
                self.ds = ds
            return

        # continue with filename
        try:
            self.ds = None
            self.read(filename, **kwargs)
        except (OSError, IOError, RuntimeError):
            try:
                self.read(filename, **kwargs)
            except Exception:
                raise IOError("Unkown file format. Known formats are netcdf.")

        self.transport = None

    def __repr__(self):
        try:
            string = "xarray dataset with {} time steps \n" \
                     "Available fields: {}".format(
                self.timesteps, ", ".join(self.variables)
            )
        except AttributeError:
            string = "Empty TEF object \n" \
                     "Hint: Have you used read() to load data?"
        return string

    def __str__(self):
        return 'Class {}: \n'.format(self.__class__.__name__, self.ds)

    def __len__(self):
        return len(self.ds)

    def __getattr__(self, attr):
        if attr in self.__dict__:
            return getattr(self, attr)
        return getattr(self.ds, attr)

    def __getitem__(self, key):
        return self.ds[key]

    @property
    def timesteps(self):
        if len(self.ds.dims) != 3:
            logger.warning(
                "\nDimensions should be equal to 3, but they are not.\n"
                "You want ... "
            )
            return self.ds.dims[self._get_name_time()]
        return self.ds.dims[self.get_name_time()]

    @property
    def variables(self):
        return list(self.ds.data_vars)

    @property
    def dimensions(self):
        return list(self.ds.dims)

    def read(self, filename,  **kwargs):
        """Reads xarray dataset"""
        if self.ds is None:
            self.ds = xr.open_dataset(filename, **kwargs)
            logger.debug("read: {}".format(self.__str__))
        else:
            raise ValueError("TEF object is already set!")

    def set_up(self,
           time_name=None,
           longitude_name=None,
           latitude_name=None,
           depth_name=None):
        """Setup xarray.dataset and transposes dimensions into needed format"""
        # set dimensions
        if time_name is None:
            self._time_name = self._get_name_time()
        else:
            self._time_name = time_name
        if longitude_name is None:
            self._longitude_name = self._get_name_longitude()
            if self._longitude_name is None:
                self.ds = self.ds.expand_dims("lon")
                self._longitude_name = "lon"
                print("Created dummy dimension for longitude")
        else:
            self._longitude_name = longitude_name

        if latitude_name is None:
            self._latitude_name = self._get_name_latitude()
            if self._latitude_name is None:
                self.ds = self.ds.expand_dims("lat")
                self._latitude_name = "lat"
                print("Created dummy dimension for latitude")
        else:
            self._latitude_name = latitude_name

        if depth_name is None:
            self._depth_name = self._get_name_depth()
            if self._depth_name is None:
                self.ds = self.ds.expand_dims("depth")
                self._depth_name = "depth"
                print("Created dummy dimension for depth")
        else:
            self._depth_name = depth_name
        if time_name is None:
            self._time_name = self._get_name_time()
            if self._time_name is None:
                self.ds = self.ds.expand_dims("time")
                self._time_name = "time"
        else:
            self._time_name = time_name
        
        # Transpose data
        self.ds = self.ds.transpose(self._time_name,
                                    self._depth_name,
                                    self._latitude_name,
                                    self._longitude_name)

    def _get_name_time(self):
        """
        check for 'time' dimension and return name
        """
        # check unit
        for dim in self.ds.dims:
            if (('units' in self.ds[dim].attrs and
                'since' in self.ds[dim].attrs['units']) or
                ('units' in self.ds[dim].encoding and
                 'since' in self.ds[dim].encoding['units']) or
                dim in ['time']):
                return dim
        # check dtype
        for dim in self.ds.variables:
            try:
                var = self.ds[dim].data[0]
            except IndexError:
                var = self.ds[dim].data
            if isinstance(var, "datetime64"):
                return dim
        # no 'time' dimension found
        logger.warning(
            "\n 'time' dimension (dtype='datetime64[ns]') not found."
        )
        return None

    def _get_name_longitude(self):
        """
        check for 'longitude' dimension and return name
        """
        for dim in self.ds.dims:
            if (('units' in self.ds[dim].attrs and
               self.ds[dim].attrs['units'] in ['degree_east', 'degrees_east']) or
               dim in ['lon', 'longitude', 'x']):
                   return dim
        # no 'longitude' dimension found
        logger.warning(
            "\n 'longitude' dimension (unit='degrees_east') not found."
        )
        return None


    def _get_name_latitude(self):
        """
        check for 'latitude' dimension and return name
        """
        for dim in self.ds.dims:
            if (('units' in self.ds[dim].attrs  and
                self.ds[dim].attrs['units'] in ['degree_north', 'degrees_north']) or
                dim in ['lat', 'latitude', 'y']):
                return dim
        # no 'latitude' dimension found
        logger.warning(
            "\n 'latitude' dimension (unit='degrees_north') not found."
        )
        return None

    def _get_name_depth(self):
        """
        check for 'latitude' dimension and return name
        """
        for dim in self.ds.dims:
            if (('units' in self.ds[dim].attrs  and
                self.ds[dim].attrs['units'] in ['vertical', 'level']) or
                dim in ['level', 'depth']):
                return dim
        # no 'latitude' dimension found
        logger.warning(
            "\n 'depth not found"
        )
        return None

    def calc_volume_transport(self, vel, height, delta):
        self.volume_transport = vel*height*delta
    
    def calc_salt_transport(self, salt, vel, height, delta):
        self.salt_transport = salt*vel*height*delta

    def sort_1dim(self, sort_by_variable = None, transport = None, N = None, minmaxrange = None):
        if sort_by_variable is None:
            raise ValueError("Please define a variable that you want to sort by.")
        if transport is None:
            raise ValueError("Please provided transport term or calculate transport by using calc_volume_transport()")

        if minmaxrange is None:
            varmin = np.floor(sort_by_variable.min().values)
            varmax = np.ceil(sort_by_variable.max().values)
            logger.info("Using minimum and maximum of the data to construct sorting array")
        else:
            if minmaxrange[0] > sort_by_variable.min().values:
                print("Warning: Given minimum value is greater than the minimum value of the variable.")
                print("Warning: Given {}, minmum value of variable {}".format(minmaxrange[0],
                                                                     sort_by_variable.min().values))
            if minmaxrange[-1] < sort_by_variable.max().values:
                print("Warning: Given maximum value is smaller than the maximum value of the variable.")
                print("Warning: Given {}, maximum value of variable {}".format(minmaxrange[-1],
                                                                     sort_by_variable.max().values))
            if type(minmaxrange) != "numpy.ndarray" and type(minmaxrange) is not tuple:
                raise ValueError("Please provide array range, e.g. np.arange(0,10), or a tuple, e.g. (0,10).")
            
            else:
                varmin = minmaxrange[0]
                varmax = minmaxrange[-1]

        if N is None:
            N = 1024
            logger.info("Setting N to default value of 1024")

        if type(minmaxrange) == "numpy.ndarray":
            logger.info('Using provided numpy array')

            var_q = minmaxrange
            
            #check if equidistant
            diff=np.diff(np.diff(var_q))
            if len(diff[diff!=0]) != 0:
                print('Warning: Provided array is not equidistant, but the function assumes equidistance!')
            delta_var=var_q[1]-var_q[0]
            
            var_Q = np.arange(var_q-0.5*delta_var, var_q[-1]+0.5*delta_var, delta_var)
        else:
            logger.info('Constructing var_q and var_Q')
            delta_var = ((varmax-varmin)/N)

            var_q = np.linspace(varmin + 0.5*delta_var,
                                    varmax - 0.5*delta_var,
                                    N)

            var_Q = np.linspace(varmin, varmax, N+1)

        idx = xr.apply_ufunc(np.digitize, sort_by_variable, var_Q) 
        
        if len(transport.time.shape)==0:
        #TODO have a better check for time axis
            logger.info('No time axis')
            out_q = np.zeros((N,),dtype=np.float64)
            
            for i in range(N):
                
                out_q[i] = transport.where(idx == i).sum([self._get_name_depth(),
                                                     self._get_name_latitude(),
                                                     self._get_name_longitude()],dtype=np.float64)/ delta_var 
                
            out_Q = np.zeros((N+1,),dtype=np.float64)

            for i in range(N):
                out_Q[i] = np.sum(out_q[i:] * delta_var)


            out = xr.Dataset({
            "q": (["var_q"], out_q),
            "Q": (["var_Q"], out_Q)},
            coords={
                "var_q": (["var_q"],var_q),
                "var_Q": (["var_Q"], var_Q),
            })

            
        else:
            logger.info('Time axis exists, will be kept')
            out_q = np.zeros((self.timesteps,N))

            for i in range(N):
                out_q[:, i] = transport.where(idx == i).sum([self._get_name_depth(),
                                                     self._get_name_latitude(),
                                                     self._get_name_longitude()],dtype=np.float64) / delta_var

            out_Q = np.zeros((self.timesteps, N+1))

            for i in range(N):
                out_Q[:, i] = np.sum(out_q[:, i:] * delta_var)


            out = xr.Dataset({
            "q": (["time", "var_q"], out_q),
            "Q": (["time", "var_Q"], out_Q)},
            coords={
                "time": (["time"], self.ds[self._get_name_time()]),
                "var_q": (["var_q"],var_q),
                "var_Q": (["var_Q"], var_Q),
            })

        return out


    def sort_2dim(self, sort_by_variable = None,
                        sort_by_variable2 = None,
                        transport = None,
                        N = None,
                        minmaxrange = None,
                        minmaxrange2 = None):
            """Sort transport by two given variables"""
            if sort_by_variable is None:
                raise ValueError("Please define a variable that you want to sort by.")
            if sort_by_variable2 is None:
                raise ValueError("Please define a second variable that you want to sort by.")    

            if transport is None:
                raise ValueError("Please provided transport term or calculate transport by using calc_transport()")

            if minmaxrange is None:
                varmin = np.floor(sort_by_variable.min().values)
                varmax = np.ceil(sort_by_variable.max().values)      
            else:
                if minmaxrange[0] > sort_by_variable.min().values:
                    print("Warning: Given minimum value is gretaer than the minimum value of the variable.")
                    print("Warning: Given {}, minmum value of variable {}".format(minmaxrange[0],
                                                                         sort_by_variable.min().values))
                if minmaxrange[-1] < sort_by_variable.max().values:
                    print("Warning: Given maximum value is smaller than the maximum value of the variable.")
                    print("Warning: Given {}, maximum value of variable {}".format(minmaxrange[-1],
                                                                         sort_by_variable.max().values))
                if type(minmaxrange) != "numpy.ndarray" and type(minmaxrange) is not tuple:
                    raise ValueError("Please provide array range, e.g. np.arange(0,10), or a tuple, e.g. (0,10).")
                else: 
                    logger.info("minmaxrange is a tuple")
                    varmin = minmaxrange[0]
                    varmax = minmaxrange[-1]

            if minmaxrange2 is None:
                varmin2 = np.floor(sort_by_variable2.min().values)
                varmax2 = np.ceil(sort_by_variable2.max().values)
            else:
                if minmaxrange2[0] > sort_by_variable2.min().values:
                    print("Warning: Given minimum value is greater than the minimum value of the variable.")
                    print("Warning: Given {}, minmum value of variable {}".format(minmaxrange2[0],
                                                                         sort_by_variable2.min().values))
                if minmaxrange2[-1] < sort_by_variable2.max().values:
                    print("Warning: Given maximum value is smaller than the maximum value of the variable.")
                    print("Warning: Given {}, maximum value of variable {}".format(minmaxrange2[-1],
                                                                         sort_by_variable2.max().values))
                if type(minmaxrange2) != "numpy.ndarray" and type(minmaxrange2) is not tuple:
                    raise ValueError("Please provide array range, e.g. np.arange(0,10), or a tuple, e.g. (0,10).")
                else:    
                    logger.info("minmaxrange2 is a tuple")
                    varmin2 = minmaxrange2[0]
                    varmax2 = minmaxrange2[-1]

            if N is None:
                N = 1024
                logger.info("Setting N to default value of 1024")

            if type(minmaxrange) == "numpy.ndarray":
                logger.info('Using provided numpy array for variable 1')

                var_q = minmaxrange

                #check if equidistant
                diff=np.diff(np.diff(var_q))
                if len(diff[diff!=0]) != 0:
                    print('Warning: Provided array for variable1 is not equidistant, but the function assumes equidistance!')
                delta_var=var_q[1]-var_q[0]

                var_Q = np.arange(var_q-0.5*delta_var, var_q[-1]+0.5*delta_var, delta_var)
            else:
                #constructing
                delta_var = ((varmax-varmin)/N)           
                var_q = np.linspace(varmin + 0.5*delta_var,
                                    varmax - 0.5*delta_var,
                                    N)                           
                var_Q = np.linspace(varmin, varmax, N+1)
                
            if type(minmaxrange2) == "numpy.ndarray":
                logger.info('Using provided numpy array for variable 2')

                var_q2 = minmaxrange2

                #check if equidistant
                diff=np.diff(np.diff(var_q2))
                if len(diff[diff!=0]) != 0:
                    print('Warning: Provided array for variable2 is not equidistant, but the function assumes equidistance!')
                delta_var2=var_q2[1]-var_q2[0]
        
                var_Q2 = np.arange(var_q2-0.5*delta_var2, var_q2[-1]+0.5*delta_var2, delta_var2)
            else:
                #contructing
                delta_var2 = ((varmax2-varmin2)/N)
                var_q2 = np.linspace(varmin2 + 0.5*delta_var2,
                                     varmax2 - 0.5*delta_var2,
                                     N)
                var_Q2 = np.linspace(varmin2, varmax2, N+1)
            
            #sortingt
            idx = xr.apply_ufunc(np.digitize, sort_by_variable, var_Q)
            idy = xr.apply_ufunc(np.digitize, sort_by_variable2, var_Q2)

            
            
            if len(transport.time.shape)==0:
                #TODO have a better check for time axis
                logger.info('No time axis')
                
                out_q = np.zeros((N, N))
                
                for i in range(N):
                    for j in range(N):
                        out_q[i, j] = transport.where((idx == i) & (idy == j)).sum([self._get_name_depth(),
                                                                                  self._get_name_latitude(),
                                                                                  self._get_name_longitude()],dtype=np.float64) / delta_var / delta_var2

                out_Q = np.zeros((N+1, N+1))

                for i in range(N):
                    for j in range(N):
                        out_Q[i, j] = np.sum(out_q[i:, j:] * delta_var * delta_var2)


                out = xr.Dataset({
                "q2": (["var_q", "var_q2"], out_q),
                "Q2": (["var_Q", "var_Q2"], out_Q)},
                coords={
                    "var_q": (["var_q"],var_q),
                    "var_q2": (["var_q2"], var_q2),
                    "var_Q": (["var_Q"], var_Q),
                    "var_Q2": (["var_Q2"], var_Q2),
                })
                
            else:
                logger.info('Time axis exists, will be kept')
                
                out_q = np.zeros((self.timesteps,N, N))
                
                for i in range(N):
                    for j in range(N):
                        out_q[:, i, j] = transport.where((idx == i) & (idy == j)).sum([self._get_name_depth(),
                                                                                  self._get_name_latitude(),
                                                                                  self._get_name_longitude()],dtype=np.float64) / delta_var / delta_var2

                out_Q = np.zeros((self.timesteps, N+1, N+1))

                for i in range(N):
                    for j in range(N):
                        out_Q[:, i, j] = np.sum(out_q[:, i:, j:] * delta_var * delta_var2)


                out = xr.Dataset({
                "q2": (["time", "var_q", "var_q2"], out_q),
                "Q2": (["time", "var_Q", "var_Q2"], out_Q)},
                coords={
                    "time": (["time"], self.ds[self._get_name_time()]),
                    "var_q": (["var_q"],var_q),
                    "var_q2": (["var_q2"], var_q2),
                    "var_Q": (["var_Q"], var_Q),
                    "var_Q2": (["var_Q2"], var_Q2),
                })

            return out
        
    def calc_bulk_values(self,
                         coord,
                         Q,
                         Q_thresh=None,
                         **kwargs):
        
        coord_min=coord[0]
        delta_var=coord[1]-coord[0]

        if Q_thresh is None:
            #set a default thresh
            Q_thresh=0.01*np.max(np.abs(Q))

        if len(Q.shape) > 1:
            #first dimension is time! -> keep this dimension!
            #prepare storage arrays for Qin, Qout, consider multiple inflow/outflows! 
            
            Qin_ar = np.zeros((Q.shape[0],10)) #10 is the dummy length
            Qout_ar = np.zeros((Q.shape[0],10))
            divval_ar = np.zeros((Q.shape[0],11)) #if there are 10 transports there would be 11 dividing salinities

            for t in np.arange(Q.shape[0]):
                ind,minmax = self._find_extrema(Q[t],Q_thresh)

                div_val=[]
                i=0
                for i in range(len(ind)):
                    div_val.append(coord_min+delta_var*ind[i])
                    i+=1
                    #calculate transports etc.
                Q_in_m=[]
                Q_out_m=[]
                index=[]
                i=0
                for i in range(len(ind)-1):
                    Q_i=-(Q[ind[i+1]]-Q[ind[i]])
                    if Q_i<0:
                        Q_out_m.append(Q_i)
                    elif Q_i > 0:
                        Q_in_m.append(Q_i)
                    else:
                        index.append(i)
                    i+=1
                div_val = np.delete(div_val, index)

                #storing results
                for i,qq in enumerate(Q_in_m):
                    Qin_ar[t,i] = qq
                for i,qq in enumerate(Q_out_m):
                    Qout_ar[t,i] = qq
                for i,ss in enumerate(div_sal):
                    divval_ar[t,i] = ss
        
            #create a xarray Dataset for the results
            out = xr.Dataset(
            {
                "Qin": (["time", "m"], np.array(Qin_ar)),
                "Qout": (["time", "n"], np.array(Qout_ar)),
                "divval": (["time", "o"], np.array(divval_ar)),
            },
            coords={
                "time": (["time"],self.ds[self._get_name_time()]),
                "m": (["m"],np.arange(len(Qin_ar))),
                "n": (["n"],np.arange(len(Qout_ar))),
                "o": (["o"],np.arange(len(divval_ar))),
            },
            )

        else:
            #no time axis
            ind,minmax = self._find_extrema(Q,Q_thresh)
            div_val=[]
            i=0
            while i < len(ind):
                    #print(Qvl[ind[i]])
                div_val.append(coord_min+delta_var*ind[i])
                i+=1
                    #print(smin+dss*ind[i])
                #calculate transports etc.
            Q_in_m=[]
            Q_out_m=[]
            index=[]
            i=0
            for i in range(len(ind)-1):
                Q_i=-(Q[ind[i+1]]-Q[ind[i]])
                if Q_i<0:
                    Q_out_m.append(Q_i)
                elif Q_i > 0:
                    Q_in_m.append(Q_i)
                else:
                    index.append(i)
                i+=1
            div_val = np.delete(div_val, index)
            
            out = xr.Dataset(
            {
                "Qin": (["m"], np.array(Q_in_m)),
                "Qout": (["n"], np.array(Q_out_m)),
                "divval": (["o"], np.array(div_val)),
            },
            coords={
                "m": (["m"],np.arange(len(Q_in_m))),
                "n": (["n"],np.arange(len(Q_out_m))),
                "o": (["o"],np.arange(len(div_val))),
            }
            )
        return(out)
        
        
    def _find_extrema(self,x,min_transport):
        """
        internal function called by calc_bulk values to find the extrema in the transport function x
        and label them correctly
        x: Q(S)
        min_transport: Q_thresh
        """
        if np.count_nonzero(x)==0:
            indices=[0]
            minmax=[0]
            return(indices,minmax)
        else:
            ###
            #set a minimum value to get rid of numerical noise
            ###
            if min_transport<=10**(-10):
                min_transport=10**(-10)

            ####
            #finding all extrema by evaluating each data point
            ####
            comp=1
            indices = []
            minmax = []
            i = 0
            while i < np.shape(x)[0]:
                if i-comp < 0:
                    a = 0
                else:
                    a=i-comp
                if i+comp+1>=len(x):
                    b=None
                    #c=i
                else:
                    b=i+comp+1
                    #c=b
                if x[i] == np.max(x[a:b]) and np.max(x[a:b]) != np.min(x[a:b]):# and x[i] != x[a]:
                    indices.append(i)
                    minmax.append('max')
                elif x[i] == np.min(x[a:b]) and np.max(x[a:b]) != np.min(x[a:b]):# and (x[i] != x[c] or x[i] != x[a]):
                    indices.append(i)
                    minmax.append('min')
                i+=1
            #print(indices,minmax)
            #print(x[indices])

            ###
            #correct consecutive extrema of the same kind, e.g., min min min or max max max (especially in the beginning and end of the salinity array)
            ###

            #index=[]
            ii=1
            while ii < len(indices):
                #print('minmin/maxmax',ii,len(indices))
                #print(minmax)
                index=[]
                if minmax[ii] == minmax[ii-1]:
                    if minmax[ii] == 'max': #note the index of the smaller maximum
                        if x[indices[ii]]>=x[indices[ii-1]]:
                            index.append(ii-1)
                        else:
                            index.append(ii)
                    elif minmax[ii] == 'min': #note the index of the greater minimum
                        if x[indices[ii]]<=x[indices[ii-1]]:
                            index.append(ii-1)
                        else:
                            index.append(ii)
                    minmax = np.asarray(minmax)
                    indices = np.asarray(indices)
                    indices = np.delete(indices, index)
                    minmax = np.delete(minmax, index)
                else:
                    ii+=1
            #print(indices,minmax)

            ####
            #delete too small transports
            ####

            #print(indices,minmax)
            ii=0
            while ii < len(indices)-1: 
                index=[]
                #print('min_trans',ii,len(indices))
                #print(indices,minmax)
                #print(np.abs(np.abs(x[indices[ii+1]])-np.abs(x[indices[ii]])),min_transport,indices[ii],indices[ii+1])
                if np.abs(x[indices[ii+1]]-x[indices[ii]]) < min_transport:
                    #print(np.abs(x[indices[ii+1]]-x[indices[ii]]),min_transport,indices[ii],indices[ii+1],x[indices[ii]],x[indices[ii+1]])
                    if ii == 0: #if smin is involved and the transport is too small, smin has to change its min or max property
                        #print('if')
                        index.append(ii+1)
                        if minmax[ii] == 'min':
                            minmax[ii] = 'max'
                        else:
                            minmax[ii] = 'min'
                    elif ii+1==len(indices)-1:#if smax is involved and the transport is too small, smin has to change its min or max property
                        #print('elif')
                        index.append(ii)
                        if minmax[ii+1] == 'min':
                            minmax[ii+1] = 'max'
                        else:
                            minmax[ii+1] = 'min'
                    else: #else both involved div sals are kicked out
                        #print('else')
                        if ii+2 < len(indices)-1:
                        #check and compare to i+2
                            if minmax[ii]=='min':
                                if x[indices[ii+2]]>x[indices[ii]]:
                                    index.append(ii+2)
                                    index.append(ii+1)
                                else:
                                    index.append(ii)
                                    index.append(ii+1)
                            elif minmax[ii]=='max':
                                if x[indices[ii+2]]<x[indices[ii]]:
                                    index.append(ii+2)
                                    index.append(ii+1)
                                else:
                                    index.append(ii)
                                    index.append(ii+1)
                        else:
                            index.append(ii)
                            index.append(ii+1)
                    #print(index)
                    indices = np.delete(indices, index)
                    minmax = np.delete(minmax, index)
                    #print('after delete',indices,minmax)
                else:
                    ii+=1

            ###
            #so far the first and last minmax does not correspond to smin and smax of the data, expecially smin due to numerical errors (only makes sense)
            #correct smin index
            ###

            #print(indices,minmax)
            if len(x)>4:
                ii=1
                while np.abs(np.abs(x[ii])-np.abs(x[0])) < 10**(-10) and ii < len(x)-1:
                    ii+=1
                indices[0]=ii-1
                #correct smax index
                if x[-1]==0: #for low salinity classes Q[-1] might not be zero as supposed.
                    jj=-1
                    while x[jj] == 0 and np.abs(jj) < len(x)-1:
                        jj -=1
                    indices[-1] = len(x)+jj+1
            return indices,minmax

In [ ]:
ds = xr.open_dataset("data/SoH_2_2011_01.nc")#.isel(time=0)

tef = TEF_object(ds = ds)

In [ ]:
tef.ds.velx3d

<xarray.DataArray 'velx3d' (time: 744, level: 40, latc: 51, lonc: 1)>
[1517760 values with dtype=float32]
Coordinates:
  * latc     (latc) float64 26.37 26.38 26.4 26.42 ... 27.15 27.17 27.18 27.2
  * level    (level) float64 1.0 2.0 3.0 4.0 5.0 ... 36.0 37.0 38.0 39.0 40.0
  * lonc     (lonc) float64 56.38
  * time     (time) datetime64[ns] 2011-01-01T01:00:00 ... 2011-02-01
Attributes:
    units:         m/s
    long_name:     velocity in global x-direction (3D)
    cell_methods:  time: mean

All variables have to be in the xarray dataset. 
variable konsistent benennen
schichtdicke nicht depth

In [ ]:
show_doc(TEF_object)

<h2 id="TEF_object" class="doc_header"><code>class</code> <code>TEF_object</code><a href="" class="source_link" style="float:right">[source]</a></h2>

> <code>TEF_object</code>(**`filename`**=*`None`*, **`ds`**=*`None`*, **\*\*`kwargs`**)

Total Exchange Flow (TEF) Object used for calculating TEF properties

In [ ]:
show_doc(TEF_object.read)

<h4 id="TEF_object.read" class="doc_header"><code>TEF_object.read</code><a href="__main__.py#L69" class="source_link" style="float:right">[source]</a></h4>

> <code>TEF_object.read</code>(**`filename`**, **\*\*`kwargs`**)

Reads xarray dataset